# Week 3 — Part 02: Structured Prompt Specification

**Estimated time:** 60–90 minutes

---

## Pre-study (Self-learn)

Foundations Course assumes Self-learn is complete. If you need a refresher:

- [Foundations Course Pre-study index](../PRESTUDY.md)
- [Self-learn — Prompt engineering and evaluation](../self_learn/Chapters/3/02_prompt_engineering_evaluation.md)
- [Self-learn — Structured outputs and schemas](../self_learn/Chapters/3/01_function_calling_structured_outputs.md)

---

## What success looks like (end of Part 02)

- You can write a prompt that acts like a contract: explicit keys, explicit constraints, and clear handling for missing data.
- You can validate a model's output deterministically (parse + schema checks).

### Checkpoint

- You can run `parse_contract_output(...)` on at least 2 simulated outputs.
- Your prompt explicitly bans extra keys and non-JSON text.

## Learning Objectives

- Treat prompts as specifications (contracts)
- Define clear preconditions and postconditions
- Design strict JSON output contracts
- Create validators that make outputs checkable

### What this part covers
This notebook teaches you to treat prompts as **API contracts** — not as "clever wording" but as formal specifications with:
- a defined input format
- an exact output schema (JSON keys, types)
- explicit constraints (no extra keys, no markdown, no prose)
- fallback conditions (what to return when the answer is unknown)

**Why this matters:** A vague prompt produces unpredictable output. A contract prompt produces output you can validate programmatically — which is essential for building reliable pipelines.

## What is a Prompt?

A **prompt** is the input text or instructions you send to a Large Language Model (LLM). It acts as the API contract between your code and the AI.

Typically, when using an LLM API (like OpenAI's), a prompt is broken down into structured roles:
- **System**: High-level instructions, persona, and rules (e.g., "You are a helpful Python expert. Always return JSON").
- **User**: The specific request, task, or data the user wants processed.

Let's look at a basic example of how to construct and send a prompt using the OpenAI API format.

In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

# Make sure you have a .env file with your OPENAI_API_KEY
load_dotenv()

# We initialize the client
client = OpenAI()

def call_llm(system_prompt: str, user_prompt: str) -> str:
    """
    A basic wrapper around an LLM API call demonstrating 'What is a Prompt?'.
    We separate the 'System' (rules/persona) from the 'User' (task/data).
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0 # Keep it deterministic
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error calling API: {e}"

# Example of a System Prompt (the "Contract")
system_prompt = """
You are a helpful Python expert. 
Your goal is to explain concepts clearly to beginners.
"""

# Example of a User Prompt (the "Input")
user_prompt = "Explain what a Python dictionary is in one sentence."

# Let's run it
output = call_llm(system_prompt, user_prompt)
print("=== LLM Response ===")
print(output)

## Overview

In this lab, you will practice writing prompts as **checkable specs**.

You will:

- write a strict JSON “contract prompt”
- validate outputs deterministically (parse + exact key checks)
- extend the contract with a refusal/error shape (TODO)

If you need the conceptual background, use the Self-learn links at the top of the notebook.

### What this cell does
Defines the core validation infrastructure:

- **`Extracted` dataclass** — the typed output schema. `person` and `company` are both `Optional[str]` — they can be `null` when the model can't find them.
- **`validate_exact_keys()`** — checks that the JSON has *exactly* the required keys: no missing, no extra. Extra keys are a contract violation just as much as missing keys.
- **`parse_contract_output()`** — the full parse + validate pipeline: JSON parse → key check → type check → typed result.

**Key insight:** Separating parse from validate from type-check gives you precise error messages. "Missing keys: ['company']" tells you exactly what went wrong and where to fix the prompt.

In [ ]:
import json
from dataclasses import dataclass
from typing import Optional, Set


@dataclass(frozen=True)
class Extracted:
    person: Optional[str]
    company: Optional[str]


def validate_exact_keys(obj: dict, required_keys: Set[str]) -> None:
    extra = set(obj.keys()) - required_keys
    missing = required_keys - set(obj.keys())
    if missing:
        raise ValueError(f"missing keys: {sorted(missing)}")
    if extra:
        raise ValueError(f"extra keys: {sorted(extra)}")


def parse_contract_output(text: str) -> Extracted:
    data = json.loads(text)
    if not isinstance(data, dict):
        raise ValueError("output must be a JSON object")
    validate_exact_keys(data, {"person", "company"})

    person = data.get("person")
    company = data.get("company")

    if person is not None and not isinstance(person, str):
        raise ValueError("person must be string or null")
    if company is not None and not isinstance(company, str):
        raise ValueError("company must be string or null")

    return Extracted(person=person, company=company)


print(parse_contract_output('{"person": "Ada", "company": null}'))

### What this cell does
Defines `build_extraction_prompt()` — a function that constructs a strict contract prompt from input text.

**Notice the structure of the prompt:**
1. **Role** — tells the model what it is doing
2. **Task** — what to produce
3. **Output format** — ONLY JSON, exact keys listed
4. **Rules** — no markdown, no extra keys, use null when missing
5. **Input** — the actual text to process

**Why explicit rules matter:** Without "no markdown", the model might wrap the JSON in a code block (` ```json ... ``` `). Without "use null when missing", it might hallucinate a value. Each rule closes a specific failure mode.

## Exercise: Write a contract prompt

Below is a contract prompt template. Your job is to adjust it for new tasks while keeping it checkable.

Key properties:

- “Return ONLY valid JSON”
- “exactly these keys”
- “Use null if not found”

### What this cell does
Defines `simulate_model_output()` — a stand-in for a real LLM call — then runs the full contract pipeline: build prompt → simulate output → parse and validate.

**Why use a simulator?** You can test your parsing and validation logic without making real API calls (which cost money and require a key). Once the contract logic is solid, swapping in a real LLM call is a one-line change.

**Your task:** Implement `build_refusal_contract_todo()` — extend the contract to return either `{"ok": true, ...}` or `{"ok": false, "error": "..."}`. This "refusal shape" is important: it lets downstream code distinguish "extraction succeeded" from "extraction was impossible" without relying on null-checking heuristics.

In [ ]:
def build_extraction_prompt(text: str) -> str:
    # Non-verbatim: same idea, slightly different wording and ordering.
    return (
        "Role: Information extraction engine.\n"
        "Task: Extract a person name and a company name.\n"
        "Output: ONLY JSON with keys person, company.\n"
        "Rules: no markdown, no additional keys, use null when missing.\n\n"
        f"Input text:\n{text}\n"
    )


print(build_extraction_prompt("Ada Lovelace founded nothing."))

In [ ]:
def simulate_model_output(prompt: str) -> str:
    # Simulated outputs for practice (stand-in for real LLM call)
    if "Ada" in prompt:
        return '{"person": "Ada Lovelace", "company": null}'
    return '{"person": null, "company": null}'


raw = simulate_model_output(build_extraction_prompt("Ada Lovelace wrote notes."))
print("raw:", raw)
print("parsed:", parse_contract_output(raw))

In [ ]:
def build_refusal_contract_todo(text: str) -> str:
    """TODO: extend the contract to return either:

    - {"ok": true, "person": ..., "company": ...}
    - OR {"ok": false, "error": "..."}

    Keep it strict:

    - return ONLY JSON
    - no additional keys
    - use null for missing fields when ok=true
    """
    return (
        "Role: Information extraction engine.\n"
        "Task: Extract a person name and a company name.\n"
        "Output: ONLY JSON with exactly these keys: ok, person, company, error.\n"
        "Rules: no markdown, no additional keys.\n"
        "Rules: if extraction is possible, set ok=true, set error=null.\n"
        "Rules: if extraction is not possible, set ok=false, set person=null, company=null, and set error to a short reason.\n\n"
        f"Input text:\n{text}\n"
    )


print(build_refusal_contract_todo("Ada Lovelace wrote notes."))

## Self-check

- Does your prompt define exact keys?
- Does it forbid extra text?
- Does it define what to do when info is missing?

## References

- Prompting guide: https://www.promptingguide.ai/
- Anthropic cookbook: https://github.com/anthropics/anthropic-cookbook

## Appendix: Solutions (peek only after trying)

Reference implementation for `build_refusal_contract_todo`.

In [ ]:
def build_refusal_contract_todo(text: str) -> str:
    return (
        "Role: Information extraction engine.\n"
        "Task: Extract a person name and a company name.\n"
        "Return ONLY valid JSON.\n"
        "Output schema (exact keys): ok, person, company, error\n"
        "Constraints: no markdown, no prose, no extra keys.\n"
        "If a value is missing, use null.\n"
        "If you cannot comply, return ok=false and a short error message.\n\n"
        f"INPUT:\n{text}\n"
    )


print(build_refusal_contract_todo("No names here."))